<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Alexander Kollmann, 10/2022**

---

**Funktion**

Verbindung zu einer MySQL-DB herstellen

CRUD-Operationen ausführen

SQL-Dump der Demodatenbank ganz unten

---



**Referenzen**

https://www.w3schools.com/sql/default.asp



---

In [1]:
# Notwendige Bibliotheken installieren um MySQL-Datenbank ansprechen zu können
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 204 kB/s 


In [2]:
# Importieren der notwendiggen Bibliotheken
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [3]:
# Verbindung zu der Datenbank aufbauen. Bei dem Beisiel handelt es sich um eine Datenbank auf einem freien SQL-Server. D.h. KEINE persönliche Information abspeichern.

#MYSQL_HOSTNAME = 'sql11.freemysqlhosting.net'

MYSQL_HOSTNAME = 'remotemysql.com'
MYSQL_USER = 'ddv67SeI3z'
MYSQL_PASSWORD = 'i934PvTXFJ'
MYSQL_DATABASE = 'ddv67SeI3z'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'

con = create_engine(connection_string)

In [5]:
# INSERT (gerade bei DB-Operationen wird empfohlen ein Error-Handling (try-except) zu verwenden um mögliche Fehler "abzufangen")
try:
    query="INSERT INTO  `ddv67SeI3z`.`pat` (`pat_id` ,`pat_vorname` ,`pat_nachname` ,`pat_gebdat`)  VALUES('bpk5','heute','abend','2022-11-07')"
    
    id=con.execute(query)
    
    print("Rows Added  = ",id.rowcount)
    
except SQLAlchemyError as e:
        error = str(e.__dict__['orig'])
        print(error)

Rows Added  =  1


In [ ]:
# CREATE/ INSERT (mehrere)

try:
    query="INSERT INTO  `ddv67SeI3z`.`pat` (`pat_id` ,`pat_vorname` ,`pat_nachname` ,`pat_gebdat`)  VALUES(%s,%s,%s,%s)"
    data=[('bpk1','vn1','nn1','1950-12-01'),
            ('bpk2','vn2','nn2','1950-12-02'),
            ('bpk3','vn3','nn3','1950-12-03')]
    
    id=con.execute(query,data)
    
    print("Rows Added  = ",id.rowcount)
    
except SQLAlchemyError as e:
        error = str(e.__dict__['orig'])
        print(error)

(1062, "Duplicate entry 'bpk1' for key 'PRIMARY'")


In [6]:
# SELECT / READ (einfach)

import pandas as pd

query = """SELECT * FROM pat;""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_id,pat_vorname,pat_nachname,pat_gebdat
0,bp54,heute,abend,2022-11-07
1,bpk1,vn1,nn1,1950-12-01
2,bpk2,vn2,nn2,1950-12-02
3,bpk3,vn3,nn3,1950-12-03
4,bpk5,heute,abend,2022-11-07


In [8]:
# SELECT / READ (join)
# Welche Patienten haben eine "offene, stationäre Kontaktbestätigung" in einer Krankenanstalt (d.h. diese Patienten sind zzt. stationär aufgenommen, haben kein EndeDatum)
 

#Gute Erklärung der verschiedenen JOINs
# https://www.w3schools.com/sql/sql_join.asp 

import pandas as pd

query = """select `pat`.`pat_nachname` AS `pat_nachname`,`pat`.`pat_vorname` AS `pat_vorname`,`gda`.`gda_bezeichnung` AS `gda_bezeichnung` 
           from ((`kbs` join `pat` on(`kbs`.`pat_id` = `pat`.`pat_id`)) join `gda` on(`kbs`.`gda_id` = `gda`.`gda_id`)) 
           where `kbs`.`kbs_type` = 'stat' and `kbs`.`kbs_end` is null and `gda`.`gda_rolle` = 'Krankenanstalt'""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_nachname,pat_vorname,gda_bezeichnung
0,nn1,vn1,Salzburg LKA
1,nn3,vn3,Salzburg LKA


In [11]:
# UPDATE

import sqlalchemy as sa

# Reflect the database table into an object  
tbl = sa.Table('pat', sa.MetaData(), autoload_with=con)
# Create an update object
upd = sa.update(tbl).where(tbl.c.pat_id == 'bp54').values(pat_vorname='xxx', pat_nachname='yyy')

# The "begin" context manager will automatically commit on exit
with con.begin() as conn:
    conn.execute(upd)

In [ ]:
# Check - UPDATE
import pandas as pd

query = """SELECT * FROM pat;""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_id,pat_vorname,pat_nachname,pat_gebdat
0,bpk1,vn1,nn1,1950-12-01
1,bpk2,vn2,nn2,1950-12-02
2,bpk3,vn3,nn3,1950-12-03
3,bpk4,a,k,2022-02-22


In [12]:
# DELETE (Tip: Delete kann man auch über einen Parameter "ausblenden" oder "inaktiv" abbeilden)

import sqlalchemy as sa

# Reflect the database table into an object  
tbl = sa.Table('pat', sa.MetaData(), autoload_with=con)
# Create an update object
dele = sa.delete(tbl).where(tbl.c.pat_id == 'bp54')

# The "begin" context manager will automatically commit on exit
with con.begin() as conn:
    conn.execute(dele)

In [13]:
# Check - DELETE
import pandas as pd

query = """SELECT * FROM pat;""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_id,pat_vorname,pat_nachname,pat_gebdat
0,bpk1,vn1,nn1,1950-12-01
1,bpk2,vn2,nn2,1950-12-02
2,bpk3,vn3,nn3,1950-12-03
3,bpk5,heute,abend,2022-11-07


# **phpMyAdmin SQL Dump**

version 5.0.4

https://www.phpmyadmin.net/

Host: remotemysql.com

Erstellungszeit: 07. Nov 2022 um 19:36

Server-Version: 8.0.13-4

PHP-Version: 7.3.33-1+ubuntu18.04.1+deb.sury.org+1

In [ ]:


SET SQL_MODE = "NO_AUTO_VALUE_ON_ZERO";
START TRANSACTION;
SET time_zone = "+00:00";


/*!40101 SET @OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT */;
/*!40101 SET @OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS */;
/*!40101 SET @OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION */;
/*!40101 SET NAMES utf8mb4 */;

--
-- Datenbank: `ddv67SeI3z`
--

-- --------------------------------------------------------

--
-- Tabellenstruktur für Tabelle `gda`
--

CREATE TABLE `gda` (
  `gda_id` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `gda_bezeichnung` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `gda_name` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `gda_adresse` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `gda_rolle` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_german2_ci;

--
-- Daten für Tabelle `gda`
--

INSERT INTO `gda` (`gda_id`, `gda_bezeichnung`, `gda_name`, `gda_adresse`, `gda_rolle`) VALUES
('1.2.40.0.34.3.1.10113', 'Salzburg LKA', 'Landeskrankenhaus Salzburg - Universitätsklinikum ', 'Müllner Hauptstraße 48', 'Krankenanstalt'),
('1.2.40.0.34.3.2.11967', 'Dr. RICHARD BARTA', '', 'Erzherzog-Eugen-Straße 14/1', 'Ärztin/Arzt');

-- --------------------------------------------------------

--
-- Tabellenstruktur für Tabelle `kbs`
--

CREATE TABLE `kbs` (
  `kbs_id` int(11) NOT NULL,
  `pat_id` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `gda_id` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `kbs_type` char(10) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `kbs_start` date NOT NULL,
  `kbs_end` date DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_german2_ci;

--
-- Daten für Tabelle `kbs`
--

INSERT INTO `kbs` (`kbs_id`, `pat_id`, `gda_id`, `kbs_type`, `kbs_start`, `kbs_end`) VALUES
(18, 'bpk1', '1.2.40.0.34.3.2.11967', 'amb', '2022-11-04', NULL),
(19, 'bpk2', '1.2.40.0.34.3.2.11967', 'amb', '2022-10-05', NULL),
(20, 'bpk1', '1.2.40.0.34.3.1.10113', 'stat', '2022-11-04', NULL),
(21, 'bpk3', '1.2.40.0.34.3.1.10113', 'stat', '2022-11-06', NULL);

-- --------------------------------------------------------

--
-- Tabellenstruktur für Tabelle `pat`
--

CREATE TABLE `pat` (
  `pat_id` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `pat_vorname` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `pat_nachname` varchar(50) CHARACTER SET utf8mb4 COLLATE utf8mb4_german2_ci NOT NULL,
  `pat_gebdat` date NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_german2_ci;

--
-- Daten für Tabelle `pat`
--

INSERT INTO `pat` (`pat_id`, `pat_vorname`, `pat_nachname`, `pat_gebdat`) VALUES
('bpk1', 'vn1', 'nn1', '1950-12-01'),
('bpk2', 'vn2', 'nn2', '1950-12-02'),
('bpk3', 'vn3', 'nn3', '1950-12-03'),
('bpk5', 'heute', 'abend', '2022-11-07');

--
-- Indizes der exportierten Tabellen
--

--
-- Indizes für die Tabelle `gda`
--
ALTER TABLE `gda`
  ADD PRIMARY KEY (`gda_id`);

--
-- Indizes für die Tabelle `kbs`
--
ALTER TABLE `kbs`
  ADD PRIMARY KEY (`kbs_id`),
  ADD KEY `FK_kbs_pat` (`pat_id`),
  ADD KEY `FK_kbs_gda` (`gda_id`);

--
-- Indizes für die Tabelle `pat`
--
ALTER TABLE `pat`
  ADD PRIMARY KEY (`pat_id`) USING BTREE;

--
-- AUTO_INCREMENT für exportierte Tabellen
--

--
-- AUTO_INCREMENT für Tabelle `kbs`
--
ALTER TABLE `kbs`
  MODIFY `kbs_id` int(11) NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=22;

--
-- Constraints der exportierten Tabellen
--

--
-- Constraints der Tabelle `kbs`
--
ALTER TABLE `kbs`
  ADD CONSTRAINT `FK_kbs_gda` FOREIGN KEY (`gda_id`) REFERENCES `gda` (`gda_id`),
  ADD CONSTRAINT `FK_kbs_pat` FOREIGN KEY (`pat_id`) REFERENCES `pat` (`pat_id`);
COMMIT;

/*!40101 SET CHARACTER_SET_CLIENT=@OLD_CHARACTER_SET_CLIENT */;
/*!40101 SET CHARACTER_SET_RESULTS=@OLD_CHARACTER_SET_RESULTS */;
/*!40101 SET COLLATION_CONNECTION=@OLD_COLLATION_CONNECTION */;
